# Programa auxiliar para generar transformaciones al azar en los casos de cursogramas (imágenes con su correspondiente XML) aplicando técnicas de "Data Augmentation"
Basado y adaptado del código de https://medium.com/@bhuwanbhattarai/image-data-augmentation-and-parsing-into-an-xml-file-in-pascal-voc-format-for-object-detection-4cca3d24b33b

0) Configurar los parámetros para la ejecución:

In [ ]:
#@title Configurar Parámetros

#@markdown Generales:
procesar_todos_archivos_disponibles = True  #@param {type:"boolean"}
cantidad_archivos_procesar = 1500 #@param {type:"integer"}
# prefijo de archivos generados
prefijo_archivo_da = 'da' # parámetro con valor fijo

#@markdown Probabilidades de Métodos de Data Augmentation:
probab_DA_Zoom_Cambio_Tamanio = 100#@param {type:"integer"}
DA_Zoom_Cambio_Tamanio_Porcentaje_Minimo = 10 #@param {type:"slider", min:5, max:200, step:5}
DA_Zoom_Cambio_Tamanio_Porcentaje_Maximo = 100 #@param {type:"slider", min:5, max:200, step:5}
probab_DA_Estirar_Horizontal = 50 #@param {type:"integer"}
probab_DA_Estirar_Vertical = 50 #@param {type:"integer"}
probab_DA_PCA_Color = 0 #@param {type:"integer"}
probab_DA_Ruido_Gaussiano = 30 #@param {type:"integer"}
probab_DA_Ruido_Salt_and_Pepper = 50 #@param {type:"integer"}
# nota: los siguientes no tienen sentido para Cursogramas
# pero se dejan para que quede completo
probab_DA_Volteo_Vertical = 0 #-no usado- @param {type:"integer"}
probab_DA_Volteo_Horizontal = 0 #-no usado- @param {type:"integer"}
probab_DA_Rotacion = 0 #-no usado- @param {type:"integer"}

# define variables a usar luego en base a parámetros
listaMetodosDA = ['z', 'pca', 'ng', 'nsp', 'fv', 'fh', 'r', 'eh', 'ev']
listaProbabDA = [probab_DA_Zoom_Cambio_Tamanio, probab_DA_PCA_Color, probab_DA_Ruido_Gaussiano, probab_DA_Ruido_Salt_and_Pepper, probab_DA_Volteo_Vertical, probab_DA_Volteo_Horizontal, probab_DA_Rotacion, probab_DA_Estirar_Horizontal, probab_DA_Estirar_Vertical ]
if probab_DA_Zoom_Cambio_Tamanio > 0:
  if DA_Zoom_Cambio_Tamanio_Porcentaje_Minimo > DA_Zoom_Cambio_Tamanio_Porcentaje_Maximo:
    DA_Zoom_Cambio_Tamanio_Porcentaje_Maximo = DA_Zoom_Cambio_Tamanio_Porcentaje_Minimo 

#@markdown Parámetros del Drive:

# directorios locales donde se guardan las imágenes a procesar y los XMLs (pueden ser el mismo o distinto)
img_path = '/content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados' #@param {type:"string"}
xml_path = '/content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados' #@param {type:"string"}

forzar_drive_actualizar = True  #@param {type:"boolean"}

print ("Parámetros definidos.")

Parámetros definidos.


1) Cargar librerías:

In [ ]:
#@title Cargar Librerías
import os
import cv2
import xml.etree.cElementTree as ET
from PIL import Image
import numpy as np
import shutil

import random
from random import choices, randint, randrange

print ("Librerías cargadas.")

Librerías cargadas.


2) Montar el Drive:

In [ ]:
#@title Montar Google Drive
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


3) Realizar Data Augmentation:

In [ ]:
#@title Definir funciones auxiliares para Data Augmentation

# función auxiliar para aplicar zoom (cambio de tamaño) en la imagen
def da_zoom(img, porc=0):
  if (porc<=0 or porc==100):
    return img
  else:
    return da_cambiarTamanio(img, porc, porc)


# función auxiliar para cambiar tamaño de la imagen 
# usando diferentes % para alto y ancho
def da_cambiarTamanio(img, porcAncho=0, porcAlto=0):
  if (porcAncho<=0 or porcAlto<=0):
    return img
  # calcula nuevo tamaño
  alto, ancho, ch = img.shape
  alto = int(float(alto * porcAlto) / 100.0)
  ancho = int(float(ancho * porcAncho) / 100.0)
  # aplica la transformación
  imgPIL = Image.fromarray((img).astype(np.uint8))
  img_out = imgPIL.resize((ancho,alto), Image.ANTIALIAS)
  img_out = np.array(img_out).astype('float32') #/ 255.0
  return img_out


# función auxiliar para aplicar PCA color augmentation
def da_pca_color(image):
    # aplica la tansformación
    assert image.ndim == 3 and image.shape[2] == 3
    assert image.dtype == np.uint8
    img = image.reshape(-1, 3).astype(np.float32)
    sf = np.sqrt(3.0/np.sum(np.var(img, axis=0)))
    img = (img - np.mean(img, axis=0))*sf 
    cov = np.cov(img, rowvar=False) # calculate the covariance matrix
    value, p = np.linalg.eig(cov) # calculation of eigen vector and eigen value 
    rand = np.random.randn(3)*0.08
    delta = np.dot(p, rand*value)
    delta = (delta*255.0).astype(np.int32)[np.newaxis, np.newaxis, :]
    img_out = np.clip(image+delta, 0, 255).astype(np.uint8)
    return img_out


# función auxiliar para aplicar ruido Gaussiano
def da_noiseG(image):
    # aplica la transformación
    row,col,ch= image.shape
    mean = 0
    var = 0.2
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy    


# función auxiliar para aplicar ruido Salt & Pepper
def da_noiseSP(image):
    # aplica la transformación
    prob = 0.05
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output    


# función auxiliar para voltear la imagen en forma vertical
def da_flip_vertical(image):
    # aplica la transformación
    f_img = cv2.flip(image, 0)
    return f_img


# función auxiliar para voltear la imagen en forma horizontal
def da_flip_horizontal(image):
    # aplica la transformación
    f_img = cv2.flip(image, 1)
    return f_img


# función auxiliar para rotar la imagen 90 grados
def da_rotate_90(image):
    # aplica la transformación
    img_transpose = np.transpose(image, (1,0,2))
    f_img = cv2.flip(img_transpose, 1)
    return f_img


# función auxiliar para rotar la imagen 180 grados
def da_rotate_180(image):
    # aplica la transformación
    f_img = cv2.flip(image, -1)
    return f_img


# función auxiliar para rotar la imagen 270 grados
def da_rotate_270(image):
    # aplica la transformación
    img_transpose_270 = np.transpose(image, (1,0,2))
    f_img = cv2.flip(img_transpose_270, 0)
    return f_img

print ("Funciones de Data Augmentation para imágenes definidas.")

Funciones de Data Augmentation para imágenes definidas.


In [ ]:
#@title Procesar archivos XMLs de cada imagen y generar transformaciones 

# levanta los xml a procesar
all_xml_array = [os.path.join(xml_path, fn) for fn in os.listdir(xml_path) if fn.endswith('.xml') and not fn.startswith(prefijo_archivo_da) ]
if not procesar_todos_archivos_disponibles:
  if len(all_xml_array) > cantidad_archivos_procesar:
    # toma una muestra al azar de acuerdo a la cantidad indicada
    all_xml_array = choices(all_xml_array,  k=cantidad_archivos_procesar )

# procesa los archivos XML
new_xml_info = []
cantXMLProc = 0
cantImgGen = 0
cantXMLGen = 0
print("\n*** Comienza procesamiento de XMLs para generar nuevas imágenes ***\n")
print("  > Cantidad de XMLs a procesar: ", len(all_xml_array), "\n")
for xml_file in all_xml_array:

    cantXMLProc = cantXMLProc + 1
    print(cantXMLProc, "- ", xml_file, ":" )

    # carga la info del XML original
    et = ET.parse(xml_file)
    element = et.getroot()
    element_objs = element.findall('object') 
    element_filename = element.find('filename').text
    img_filename = os.path.join(img_path, element_filename)
    img_split = element_filename.strip().split('.png')

    # carga la imagen
    img = cv2.imread(img_filename)
    if img is None:
      print("\t\t\t\t Error al abrir la imagen ", img_filename)
      break
    alto, ancho, ch = img.shape

    # selecciona al azar los métodos de DA que va a considerar (hasta 2)
    if procesar_todos_archivos_disponibles:
      # permita que no se aplique o hasta 3 por XML (al azar) 
      DAaplicarCant = randint(0, 3)
    else:
      # siempre aplica hasta 2 por XML (al azar)
      DAaplicarCant = randint(1, 2)
    DAaplicarMetodos = choices(listaMetodosDA, weights=listaProbabDA, k=DAaplicarCant)
    print("\t\t\t DA a aplicar: ", DAaplicarMetodos)

    # aplica los métodos correspondientes
    for mDA in DAaplicarMetodos:

      if mDA=='z':        
          # aplica cambio de tamaño en la imagen (zoom)
          DAzoomUsar = randrange(DA_Zoom_Cambio_Tamanio_Porcentaje_Minimo, DA_Zoom_Cambio_Tamanio_Porcentaje_Maximo, 5) 
          print("\t\t\t % zoom: ", DAzoomUsar)
          nuevaImg = da_zoom(img, DAzoomUsar)
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + str(DAzoomUsar) + '.png'

      elif mDA=='eh' or mDA=='ev':        
          # aplica estiramiento horizontal o vertical
          porcUsar = randrange(45, 95, 5) 
          print("\t\t\t % estiramiento ", mDA[1], ": ", porcUsar)
          if mDA=='eh':
            nuevaImg = da_cambiarTamanio(img, porcAncho=porcUsar, porcAlto=100)
          else:
            nuevaImg = da_cambiarTamanio(img, porcAncho=100, porcAlto=porcUsar)
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + str(porcUsar) + '.png'

      elif mDA=='pca':        
          # aplica la transformación PCA color
          nuevaImg = da_pca_color(img)
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '.png'

      elif mDA=='ng': 
          # aplica la transformación ruido Gaussiano
          nuevaImg = da_noiseG(img)        
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '.png'     

      elif mDA=='nsp':
          # aplica la transformación ruido Salt & Pepper
          nuevaImg = da_noiseSP(img)
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '.png'

      elif mDA=='fv':
          # aplica transformación de volteo vertical
          nuevaImg = da_flip_vertical(img)
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '.png'

      elif mDA=='fh':
          # aplica transformación de volteo horizontal 
          nuevaImg = da_flip_horizontal(img)
          nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '.png'

      elif mDA=='r':
          # aplica rotación por lo que determina al azar la cantidad de grados 
          gradosRotacion = randrange(90, 270, 90) 
          if gradosRotacion == 90:
            # aplica rotación de 90 grados 
            mDA = 'r90'
            nuevaImg = da_rotate_90(img)
            nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '90.png'        
          elif  gradosRotacion == 180:
            # aplica rotación de 180 grados 
            mDA = 'r180'
            nuevaImg = da_rotate_180(img)
            nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '180.png'           
          else:
            # aplica rotación de 270 grados 
            mDA = 'r270'
            nuevaImg = da_rotate_270(img)
            nuevaImgFN = prefijo_archivo_da + '_' + img_split[0] + '-'+ mDA + '270.png'      
      else:
        # no hace nada
        print("\t\t\t\t ¡¡Tipo DA ", mDA, " no existente!!")
        break

      # si el archivo no existe
      if os.path.isfile(os.path.join(img_path, nuevaImgFN)):
        print("\t\t\t\t ", nuevaImgFN, " ya existe, se descata transformación!!")
      else:
          # graba la nueva imagen       
          cv2.imwrite(os.path.join(img_path, nuevaImgFN), nuevaImg)
          cantImgGen = cantImgGen + 1
      
          # genera el XML de la nueva imagen   
          nuevo_xml_annotation = ET.Element('annotation')
          ET.SubElement(nuevo_xml_annotation, 'folder').text = img_path
          ET.SubElement(nuevo_xml_annotation, 'filename').text = nuevaImgFN
          ET.SubElement(nuevo_xml_annotation, 'path').text = img_path + nuevaImgFN

          nuevo_xml_source = ET.SubElement(nuevo_xml_annotation, 'source')
          ET.SubElement(nuevo_xml_source, 'database').text = 'Unknown'

          nuevo_xml_size = ET.SubElement(nuevo_xml_annotation, 'size')
          ET.SubElement(nuevo_xml_size, 'width').text = str(nuevaImg.shape[1])
          ET.SubElement(nuevo_xml_size, 'height').text = str(nuevaImg.shape[0])
          ET.SubElement(nuevo_xml_size, 'depth').text = str(nuevaImg.shape[2])

          ET.SubElement(nuevo_xml_annotation, 'segmented').text = '0'

          # procesa los elementos en el archivo XML original para generar el nuevo
          for element_obj in element_objs:

              # obtiene la información actual de la imagen
              class_name = element_obj.find('name').text 

              # obtiene info del box actual
              obj_bbox = element_obj.find('bndbox')
              x1 = int(round(float(obj_bbox.find('xmin').text)))
              y1 = int(round(float(obj_bbox.find('ymin').text)))
              x2 = int(round(float(obj_bbox.find('xmax').text)))
              y2 = int(round(float(obj_bbox.find('ymax').text)))
              
              # cambia las coordenadas del box de acuerdo al tipo de transformación aplicada
              if mDA=='pca' or mDA=='ng' or mDA=='nsp':        
                  # para la transformación PCA color, ruido Gaussiano y ruido Salt & Pepper  
                  # quedan iguales
                  f_x1 = x1
                  f_x2 = x2
                  f_y1 = y1
                  f_y2 = y2

              elif mDA=='fv':
                  # para transformación de volteo vertical, cambia
                  f_x1 = x1
                  f_y1 = alto - y2
                  f_x2 = x2
                  f_y2 = alto - y1

              elif mDA=='fh':
                  # para transformación de volteo horizontal, cambia
                  f_x1 = ancho - x2
                  f_y1 = y1
                  f_x2 = ancho - x1
                  f_y2 = y2

              elif mDA=='r90':
                  # para rotación de 90 grados, cambia
                  f_x1 = alto - y2
                  f_y1 = x1
                  f_x2 = alto - y1
                  f_y2 = x2
                
              elif mDA=='r180':
                  # para rotación de 180 grados, cambia
                  f_x1 = ancho - x2
                  f_y1 = alto - y2
                  f_x2 = ancho - x1
                  f_y2 = alto - y1   
                
              elif mDA=='r270':
                  # para rotación de 270 grados, cambia
                  f_x1 = y1
                  f_y1 = ancho - x2
                  f_x2 = y2
                  f_y2 = ancho - x1
          
              else: # elif mDA=='z' or mDA=='ev' or mDA=='eh':
                  # para zoom o estiramiento vertical/horizontal,
                  #  cambia de acuerdo a nuevo tamaño
                  f_x1 = int(x1 / ancho * nuevaImg.shape[1])
                  f_x2 = int(x2 / ancho * nuevaImg.shape[1])
                  f_y1 = int(y1 / alto * nuevaImg.shape[0])
                  f_y2 = int(y2 / alto * nuevaImg.shape[0])

              # define el nuevo box
              nuevo_xml_object = ET.SubElement(nuevo_xml_annotation, 'object')
              ET.SubElement(nuevo_xml_object, 'name').text = class_name
              ET.SubElement(nuevo_xml_object, 'pose').text = 'Unspecified'
              ET.SubElement(nuevo_xml_object, 'truncated').text = '0'
              ET.SubElement(nuevo_xml_object, 'difficult').text = '0'

              nuevo_xml_bndbox = ET.SubElement(nuevo_xml_object, 'bndbox')
              ET.SubElement(nuevo_xml_bndbox, 'xmin').text = str(f_x1)
              ET.SubElement(nuevo_xml_bndbox, 'ymin').text = str(f_y1)
              ET.SubElement(nuevo_xml_bndbox, 'xmax').text = str(f_x2)
              ET.SubElement(nuevo_xml_bndbox, 'ymax').text = str(f_y2)

          # graba el nuevo XML
          nuevoXMLfn = nuevaImgFN.replace(".png", ".xml")
          xml_tree = ET.ElementTree(nuevo_xml_annotation)
          xml_tree.write( os.path.join(xml_path, nuevoXMLfn) )
          cantXMLGen = cantXMLGen + 1

print("\n*** Fin procesamiento de de XMLs para generar nuevas imágenes ***\n\n")
print("> Imágenes/XMLs procesados: ", cantXMLProc)
print("> Imágenes nuevas generadas: ", cantImgGen)
print("> XMLs nuevas generados: ", cantXMLGen)

if forzar_drive_actualizar:
  print("\n** Forzando la actualización del drive **")
  # Fuerza la actualizacion del drive
  drive.flush_and_unmount()
  # vuelve a montar
  drive.mount('/content/gdrive', force_remount=True)
  print("**Actualización del drive terminada **")




*** Comienza procesamiento de XMLs para generar nuevas imágenes ***

  > Cantidad de XMLs a procesar:  1495 

1 -  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/c_SFeRS_00021fr.xml :
			 DA a aplicar:  ['z']
			 % zoom:  30
2 -  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/c_SFeRS_00022fr.xml :
			 DA a aplicar:  ['nsp', 'nsp', 'eh']
				  da_c_SFeRS_00022fr-nsp.png  ya existe, se descata transformación!!
			 % estiramiento  h :  60
3 -  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/c_SFeRS_00023fr.xml :
			 DA a aplicar:  []
4 -  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/c_SFeRS_00024.xml :
			 DA a aplicar:  ['nsp', 'z', 'z']
			 % zoom:  50
			 % zoom:  85
5 -  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/c_SFeRS_00025fr.xml :
			 DA a aplicar:  ['z']
			 % zoom:  60
6 -  /content/gdrive/MyDrive/GEMIS/objDetectionCursogramas/Cur

In [ ]:
#@title Borrar archivos XMLs con su imagen (por si tienen algún problema)
Confirmar_borrado_archivos_DA = False #@param {type:"boolean"}

if Confirmar_borrado_archivos_DA:

  XMLAr = [os.path.join(xml_path, fn) for fn in os.listdir(xml_path) if fn.endswith('.xml') and fn.startswith(prefijo_archivo_da) ]
  print("\n** Borrando ", len(XMLAr), "archivos XML del drive **")
  for a in XMLAr:
    os.remove( a )

  PNGAr = [os.path.join(img_path, fn) for fn in os.listdir(img_path) if fn.endswith('.png') and fn.startswith(prefijo_archivo_da) ]
  print("\n** Borrando ", len(PNGAr), "archivos PNG del drive **")
  for a in PNGAr:
    os.remove( a)

  print("\n** Borrando archivos terminado **")

  if forzar_drive_actualizar:
    print("\n** Forzando la actualización del drive **")
    # Fuerza la actualizacion del drive
    drive.flush_and_unmount()
    # vuelve a montar
    drive.mount('/content/gdrive', force_remount=True)
    print("**Actualización del drive terminada **")

# resetea la variable para no borrar por error
Confirmar_borrado_archivos_DA = False